<a href="https://colab.research.google.com/github/ThienNguyen3001/Graph-Theory-ArangoDB-Final/blob/main/Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Chuẩn bị

* Gọi môi trường

In [23]:
%%capture
!git clone https://github.com/ThienNguyen3001/Graph-Theory-ArangoDB-Final.git
!rsync -av Graph-Theory-ArangoDB-Final/ ./ --exclude=.git
!pip3 install pyarango
!pip3 install "python-arango>=5.0"

In [24]:
import json
import requests
import sys
import time

import oasis
from pyArango.connection import *
from arango import ArangoClient

* **Tạo cơ sở dữ liệu tạm thời**


In [25]:
# Retrieve tmp credentials from ArangoDB Tutorial Service
login = oasis.getTempCredentials(tutorialName='test',
                                 credentialProvider='https://tutorials.arangodb.cloud:8529/_db/_system/tutorialDB/tutorialDB')

# Connect to the temp database
conn = oasis.connect(login)
db = conn[login['dbName']]

Requesting new temp credentials.
Temp database ready to use.


* **Xem thông tin DB đã tạo**


In [26]:
print('https://{}:{}'.format(login['hostname'], login['port']))
print('Username: ' + login['username'])
print('Password: ' + login['password'])
print('Database: ' + login['dbName'])


https://tutorials.arangodb.cloud:8529
Username: TUT1ig4rzxu0xbfs7ae30qli8
Password: TUT3yssm4x22w388ppdsbqsyb
Database: TUTnpdy9a7ctjb99uvlj9dp7


In [27]:
print(type(login), login, sep='\n')

<class 'dict'>
{'dbName': 'TUTnpdy9a7ctjb99uvlj9dp7', 'username': 'TUT1ig4rzxu0xbfs7ae30qli8', 'password': 'TUT3yssm4x22w388ppdsbqsyb', 'hostname': 'tutorials.arangodb.cloud', 'port': 8529}


#2 Tạo dữ liệu AQL

* **Tạo collection "Collection" trong DB**

Các bộ sưu tập trong ArangoDB. Chúng chỉ được thao tác thông qua các ngôn ngữ Ngôn ngữ truy vấn Arango NoSQL (AQL). Các hướng dẫn bên dưới tạo ra một bộ sưu tập và có thể liên quan đến các hướng dẫn DDL của ngôn ngữ SQL. Trong ví dụ này, nhóm sẽ tạo các nhân vật trong loạt phim "*Game of Thrones*".

In [28]:
collecName = 'Characters'
db.createCollection(name=collecName)

ArangoDB collection name: Characters, id: 658741668, type: document, status: loaded

* **Thêm data bằng Insert**

In [29]:
insert_query = """
INSERT {
    "name": "Ned",
    "surname": "Stark",
    "alive": true,
    "age": 41,
    "traits": ["A","H","C","N","P"]
} INTO Characters
"""

db.AQLQuery(insert_query)

* Lấy tất cả các nhân vật trong bản ghi

In [30]:
all_characters = """
FOR c IN Characters
    RETURN c
"""

query_result = db.AQLQuery(all_characters, rawResults=True)

for doc in  query_result:
    print(doc)
    print()

{'_key': '266279408230', '_id': 'Characters/266279408230', '_rev': '_i2pMESG---', 'name': 'Ned', 'surname': 'Stark', 'alive': True, 'age': 41, 'traits': ['A', 'H', 'C', 'N', 'P']}



* **Chèn các bản ghi**

In [31]:
insert_query = """
LET data = [
    { "name": "Robert", "surname": "Baratheon", "alive": false, "traits": ["A","H","C"] },
    { "name": "Jaime", "surname": "Lannister", "alive": true, "age": 36, "traits": ["A","F","B"] },
    { "name": "Catelyn", "surname": "Stark", "alive": false, "age": 40, "traits": ["D","H","C"] },
    { "name": "Cersei", "surname": "Lannister", "alive": true, "age": 36, "traits": ["H","E","F"] },
    { "name": "Daenerys", "surname": "Targaryen", "alive": true, "age": 16, "traits": ["D","H","C"] },
    { "name": "Jorah", "surname": "Mormont", "alive": false, "traits": ["A","B","C","F"] },
    { "name": "Petyr", "surname": "Baelish", "alive": false, "traits": ["E","G","F"] },
    { "name": "Viserys", "surname": "Targaryen", "alive": false, "traits": ["O","L","N"] },
    { "name": "Jon", "surname": "Snow", "alive": true, "age": 16, "traits": ["A","B","C","F"] },
    { "name": "Sansa", "surname": "Stark", "alive": true, "age": 13, "traits": ["D","I","J"] },
    { "name": "Arya", "surname": "Stark", "alive": true, "age": 11, "traits": ["C","K","L"] },
    { "name": "Robb", "surname": "Stark", "alive": false, "traits": ["A","B","C","K"] },
    { "name": "Theon", "surname": "Greyjoy", "alive": true, "age": 16, "traits": ["E","R","K"] },
    { "name": "Bran", "surname": "Stark", "alive": true, "age": 10, "traits": ["L","J"] },
    { "name": "Joffrey", "surname": "Baratheon", "alive": false, "age": 19, "traits": ["I","L","O"] },
    { "name": "Sandor", "surname": "Clegane", "alive": true, "traits": ["A","P","K","F"] },
    { "name": "Tyrion", "surname": "Lannister", "alive": true, "age": 32, "traits": ["F","K","M","N"] },
    { "name": "Khal", "surname": "Drogo", "alive": false, "traits": ["A","C","O","P"] },
    { "name": "Tywin", "surname": "Lannister", "alive": false, "traits": ["O","M","H","F"] },
    { "name": "Davos", "surname": "Seaworth", "alive": true, "age": 49, "traits": ["C","K","P","F"] },
    { "name": "Samwell", "surname": "Tarly", "alive": true, "age": 17, "traits": ["C","L","I"] },
    { "name": "Stannis", "surname": "Baratheon", "alive": false, "traits": ["H","O","P","M"] },
    { "name": "Melisandre", "alive": true, "traits": ["G","E","H"] },
    { "name": "Margaery", "surname": "Tyrell", "alive": false, "traits": ["M","D","B"] },
    { "name": "Jeor", "surname": "Mormont", "alive": false, "traits": ["C","H","M","P"] },
    { "name": "Bronn", "alive": true, "traits": ["K","E","C"] },
    { "name": "Varys", "alive": true, "traits": ["M","F","N","E"] },
    { "name": "Shae", "alive": false, "traits": ["M","D","G"] },
    { "name": "Talisa", "surname": "Maegyr", "alive": false, "traits": ["D","C","B"] },
    { "name": "Gendry", "alive": false, "traits": ["K","C","A"] },
    { "name": "Ygritte", "alive": false, "traits": ["A","P","K"] },
    { "name": "Tormund", "surname": "Giantsbane", "alive": true, "traits": ["C","P","A","I"] },
    { "name": "Gilly", "alive": true, "traits": ["L","J"] },
    { "name": "Brienne", "surname": "Tarth", "alive": true, "age": 32, "traits": ["P","C","A","K"] },
    { "name": "Ramsay", "surname": "Bolton", "alive": true, "traits": ["E","O","G","A"] },
    { "name": "Ellaria", "surname": "Sand", "alive": true, "traits": ["P","O","A","E"] },
    { "name": "Daario", "surname": "Naharis", "alive": true, "traits": ["K","P","A"] },
    { "name": "Missandei", "alive": true, "traits": ["D","L","C","M"] },
    { "name": "Tommen", "surname": "Baratheon", "alive": true, "traits": ["I","L","B"] },
    { "name": "Jaqen", "surname": "H'ghar", "alive": true, "traits": ["H","F","K"] },
    { "name": "Roose", "surname": "Bolton", "alive": true, "traits": ["H","E","F","A"] },
    { "name": "The High Sparrow", "alive": true, "traits": ["H","M","F","O"] }
]

FOR d IN data
    INSERT d INTO Characters
"""

db.AQLQuery(insert_query)

* Truy vấn cơ bản

In [32]:
all_characters_names = """
FOR name IN Characters
    RETURN name.name
"""

query_result = db.AQLQuery(all_characters_names, rawResults=True)

# Lấy ra chỉ 10 tên
for i in range(10):
  print(query_result[i])

Ned
Robert
Jaime
Catelyn
Cersei
Daenerys
Jorah
Petyr
Viserys
Jon


In [33]:
list(query_result)

['Ned',
 'Robert',
 'Jaime',
 'Catelyn',
 'Cersei',
 'Daenerys',
 'Jorah',
 'Petyr',
 'Viserys',
 'Jon',
 'Sansa',
 'Arya',
 'Robb',
 'Theon',
 'Bran',
 'Joffrey',
 'Sandor',
 'Tyrion',
 'Khal',
 'Tywin',
 'Davos',
 'Samwell',
 'Stannis',
 'Melisandre',
 'Margaery',
 'Jeor',
 'Bronn',
 'Varys',
 'Shae',
 'Talisa',
 'Gendry',
 'Ygritte',
 'Tormund',
 'Gilly',
 'Brienne',
 'Ramsay',
 'Ellaria',
 'Daario',
 'Missandei',
 'Tommen',
 'Jaqen',
 'Roose',
 'The High Sparrow']

* **Truy vấn có điều kiện**


In [34]:
find_ned_query = """
FOR c IN Characters
    FILTER c.alive == false
    RETURN c.name
"""
query_result = db.AQLQuery(find_ned_query, rawResults=True)
for doc in  query_result:
    print(doc, sep='\n')

Robert
Catelyn
Jorah
Petyr
Viserys
Robb
Joffrey
Khal
Tywin
Stannis
Margaery
Jeor
Shae
Talisa
Gendry
Ygritte


In [35]:
find_ned_query = """
FOR c IN Characters
    FILTER c.alive == true AND c.age >= 18
    RETURN c.name
"""
query_result = db.AQLQuery(find_ned_query, rawResults=True)

for doc in  query_result:
    print(doc, sep='\n')

Ned
Jaime
Cersei
Tyrion
Davos
Brienne


In [36]:
find_ned_query = """
FOR c IN Characters
    FILTER c.alive == false AND c.age >= 18
    RETURN { Name: c.name, Surname: c.surname, Age: c.age }
"""
query_result = db.AQLQuery(find_ned_query, rawResults=True)

for doc in  query_result:
    print(doc, sep='\n')

{'Name': 'Catelyn', 'Surname': 'Stark', 'Age': 40}
{'Name': 'Joffrey', 'Surname': 'Baratheon', 'Age': 19}


In [37]:
find_ned_query = """
FOR c IN Characters
    FILTER c.surname == "Bolton"
    RETURN {Name: c.name, Surname: c.surname}
"""
query_result = db.AQLQuery(find_ned_query, rawResults=True)
for doc in  query_result:
    print(doc, sep='\n')

{'Name': 'Ramsay', 'Surname': 'Bolton'}
{'Name': 'Roose', 'Surname': 'Bolton'}


In [38]:

sample_chars_query = """
FOR c IN Characters
    LIMIT 10
    RETURN c.name
"""

query_result = db.AQLQuery(sample_chars_query, rawResults=True)

for doc in  query_result:
    print(doc, sep='\n')

Ned
Robert
Jaime
Catelyn
Cersei
Daenerys
Jorah
Petyr
Viserys
Jon


In [39]:

sample_chars_query = """
FOR c IN Characters
    SORT c.name
    LIMIT 10
    RETURN { Name: c.name, Surname: c.surname, Age: c.age }
"""

query_result = db.AQLQuery(sample_chars_query, rawResults=True)

for doc in  query_result:
    print(doc, sep='\n')

{'Name': 'Arya', 'Surname': 'Stark', 'Age': 11}
{'Name': 'Bran', 'Surname': 'Stark', 'Age': 10}
{'Name': 'Brienne', 'Surname': 'Tarth', 'Age': 32}
{'Name': 'Bronn', 'Surname': None, 'Age': None}
{'Name': 'Catelyn', 'Surname': 'Stark', 'Age': 40}
{'Name': 'Cersei', 'Surname': 'Lannister', 'Age': 36}
{'Name': 'Daario', 'Surname': 'Naharis', 'Age': None}
{'Name': 'Daenerys', 'Surname': 'Targaryen', 'Age': 16}
{'Name': 'Davos', 'Surname': 'Seaworth', 'Age': 49}
{'Name': 'Ellaria', 'Surname': 'Sand', 'Age': None}


## Tạo đồ thị

In [40]:
from pyArango.collection import Collection, Field, Edges
from pyArango.graph import Graph, EdgeDefinition

# Định nghĩa Collections
class Places(Collection):
    _fields = {
        "label": Field()
    }

class Connect(Edges):
    _fields = {
        "travelTime": Field()
    }

# Định nghĩa Graph
class TravelGraph(Graph):
    _edgeDefinitions = [
        EdgeDefinition("Connect", fromCollections=["Places"], toCollections=["Places"]),
        ]
    _orphanedCollections = []


if "Places" not in db.collections:
    db.createCollection(className="Places")

if "Connect" not in db.collections:
    db.createCollection(className="Connect")

# Tạo graph
if "TravelGraph" not in db.graphs:
    travel_graph = db.createGraph("TravelGraph")
else:
    travel_graph = db.graphs["TravelGraph"]


import json

# Đọc dữ liệu từ tệp JSON
with open("places.json", "r") as f:
    places_data = json.load(f)
    db["Places"].importBulk(places_data)


with open("connect.json", "r") as f:
    connections_data = json.load(f)
    db["Connect"].importBulk(connections_data)

print("Graph created successfully!")

Graph created successfully!


## Đường đi ngắn nhất

In [42]:
query = """
FOR p IN OUTBOUND K_SHORTEST_PATHS 'Places/Aberdeen' TO 'Places/Vancouver'
GRAPH 'TravelGraph'
    LIMIT 1
    RETURN {
        places: p.vertices[*].label,
        travelTimeTotal: SUM(p.edges[*].travelTime)
    }
"""

query_result = db.AQLQuery(query, rawResults=True)

for doc in query_result:
    print(doc, sep='\n')

In [59]:

query = """
FOR v, e, p IN 2..3 OUTBOUND "Places/London" GRAPH "TravelGraph"

  OPTIONS { uniqueVertices: "path", bfs: true}
  RETURN CONCAT_SEPARATOR(" -- ", p.vertices[*].label)
"""

query_result = db.AQLQuery(query, rawResults=True)

for doc in query_result:
    print(doc, sep='\n')

London -- Birmingham -- Carlisle
London -- York -- Edinburgh
London -- York -- Carlisle
London -- Brussels -- Cologne
London -- Birmingham -- Carlisle -- Glasgow
London -- Birmingham -- Carlisle -- York
London -- York -- Edinburgh -- Leuchars
London -- York -- Edinburgh -- Glasgow
London -- York -- Carlisle -- Glasgow
London -- York -- Carlisle -- Birmingham
